In [1]:
from importlib import reload

import datasets
import numpy as np

import src.models.components.feature_extractor_dinov2
import src.models.components.spatiotemporal_encoder

In [2]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [13]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(src_tensor, sampling_rate=25, pad_to_multiple_of=32, return_tensors="pt")

input_values = batched_feature.input_values
attention_mask = batched_feature.attention_mask

print("input_values:", input_values.shape)
print("attention_mask:", attention_mask)

input_values: torch.Size([3, 32, 768])
attention_mask: tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [24]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoderConfig

config = SpatiotemporalEncoderConfig()

model = SpatiotemporalEncoder(config)
encoded = model(input_values, attention_mask)

encoded

Wav2Vec2BaseModelOutput(last_hidden_state=tensor([[[ 1.2451, -1.2089,  2.8988,  ..., -1.0611, -0.4459,  0.7890],
         [ 0.3479, -2.0541, -0.7096,  ..., -0.0651,  0.6908, -1.6089],
         [-0.1658, -0.7949, -0.8060,  ..., -1.4347, -0.3775, -0.9328],
         ...,
         [-0.8130, -0.7006,  0.9064,  ..., -0.0196,  0.0873,  0.0962],
         [-0.4527, -2.8118,  1.3964,  ...,  0.8695,  0.4114,  0.1713],
         [-0.7009, -1.9617,  1.8312,  ...,  0.3141, -0.4419, -0.0863]],

        [[-1.3427, -1.2524,  0.9374,  ..., -0.6180,  0.6089,  1.5216],
         [-1.3661, -1.0750,  0.0046,  ...,  0.5341,  0.7426,  0.7597],
         [-1.1325, -1.1512, -0.9005,  ...,  0.0219,  0.3991,  1.1402],
         ...,
         [-0.8124, -0.6057,  0.1646,  ..., -0.7590,  0.8639,  0.0988],
         [ 0.5870, -1.5820, -0.0516,  ..., -1.4687,  0.0895, -0.7045],
         [-0.2837, -1.7806,  1.7190,  ...,  0.4727,  0.2125,  0.6542]],

        [[ 0.6117, -0.4707,  2.6703,  ..., -1.6878,  0.2260,  0.2388],
   

In [29]:
reload(src.models.components.sign_language_net)
from src.models.components.sign_language_net import SignLanguageNet


model = SignLanguageNet()
out = model.generate(inputs=input_values, attention_mask=attention_mask, max_length=100)

out

tensor([[2, 2],
        [2, 2],
        [2, 2]])